In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
	root="data",
	train=True,
	download=True,
	transform=ToTensor()
)

test_data = datasets.FashionMNIST(
	root="data",
	train=False,
	download=True,
	transform=ToTensor(),
)



In [5]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


for X, y in test_dataloader:
	print(f"Shape of X [N,C,H,W] : {X.shape}")
	print(f"Shape of y: {y.shape} {y.dtype}")
	break


Shape of X [N,C,H,W] : torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models

In [6]:
device = (
	"cuda"
	if torch.cuda.is_available()
	else "mps"
	if torch.backends.mps.is_available()
	else "cpu"
)
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
	def __init__(self):
		super().__init__()
		self.flatten = nn.Flatten()
		self.linear_relu_stack = nn.Sequential(
			nn.Linear(28*28, 512),
			nn.ReLU(),
			nn.Linear(512, 512),
			nn.ReLU(),
			nn.Linear(512, 10)
		)

	def forward(self, x):
		x = self.flatten(x)
		logits = self.linear_relu_stack(x)
		return logits

model = NeuralNetwork().to(device)
print(model)


Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing Model Parameters


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
	size = len(dataloader.dataset)
	model.train()
	for batch, (X,y) in enumerate(dataloader):
		X,y = X.to(device), y.to(device)

		# Compute prediction error
		pred = model(X)
		loss = loss_fn(pred, y)

		# Back propagation
		loss.backward()
		optimizer.step()
		optimizer.zero_grad()

		if batch % 100 == 0:
			loss, current = loss.item(), (batch + 1) * len(X)
			print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
	size = len(dataloader.dataset)
	num_batches = len(dataloader)
	model.eval()
	test_loss, correct = 0, 0
	with torch.no_grad():
		for X,y in dataloader:
			X,y = X.to(device), y.to(device)
			pred = model(X)
			test_loss += loss_fn(pred,y).item()
			correct += (pred.argmax(1) == y).type(torch.float).sum().item()
	test_loss /= num_batches
	correct /= size
	print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")


In [9]:
epochs = 30
for t in range(epochs):
	print(f"Epoch {t+1}\n ......... ")
	train(train_dataloader, model, loss_fn, optimizer)
	test(test_dataloader, model, loss_fn)
print('Done')

Epoch 1
 ......... 
loss: 2.297779 [   64/60000]
loss: 2.286355 [ 6464/60000]
loss: 2.261002 [12864/60000]
loss: 2.263024 [19264/60000]
loss: 2.244042 [25664/60000]
loss: 2.211177 [32064/60000]
loss: 2.233426 [38464/60000]
loss: 2.194184 [44864/60000]
loss: 2.189903 [51264/60000]
loss: 2.157425 [57664/60000]
Test Error: 
 Accuracy: 29.5%, Avg loss: 2.153087

Epoch 2
 ......... 
loss: 2.162963 [   64/60000]
loss: 2.155674 [ 6464/60000]
loss: 2.091150 [12864/60000]
loss: 2.114026 [19264/60000]
loss: 2.060340 [25664/60000]
loss: 1.999056 [32064/60000]
loss: 2.035468 [38464/60000]
loss: 1.954583 [44864/60000]
loss: 1.957252 [51264/60000]
loss: 1.882682 [57664/60000]
Test Error: 
 Accuracy: 55.7%, Avg loss: 1.883175

Epoch 3
 ......... 
loss: 1.914447 [   64/60000]
loss: 1.891575 [ 6464/60000]
loss: 1.764229 [12864/60000]
loss: 1.810673 [19264/60000]
loss: 1.697070 [25664/60000]
loss: 1.643867 [32064/60000]
loss: 1.671017 [38464/60000]
loss: 1.573239 [44864/60000]
loss: 1.593369 [51264/6000

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


<All keys matched successfully>

In [20]:
##### App that makes predictions
#LOading model
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
		"x"
]

model.eval()
x,y = test_data[0][0], test_data[0][1]

with torch.no_grad():
	x = x.to(device)
	pred = model(x)
	predicted, actual = classes[pred[0].argmax(0)], classes[y]
	
	print(f"Precited '{predicted}', actual : '{actual}'")


Precited 'x', actual : 'x'
